# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import os
import gmaps
import gmaps.datasets
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
clean_cities = "City_data.csv"
df = pd.read_csv(clean_cities)
df.rename(columns = {'Unnamed: 0' : 'Drop'}, inplace = True)
del df['Drop']
#
df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,tautira,PF,1.590174e+09,-17.73,-149.15,80.60,65.0,20.0,4.70
1,inhambane,MZ,1.590174e+09,-23.86,35.38,68.00,94.0,16.0,3.36
2,bengkulu,ID,1.590174e+09,-3.80,102.27,77.86,81.0,100.0,4.47
3,vardo,NO,1.590174e+09,70.37,31.11,38.62,89.0,22.0,7.63
4,kirakira,SB,1.590174e+09,-10.45,161.92,79.66,85.0,17.0,10.07
...,...,...,...,...,...,...,...,...,...
549,itarema,BR,1.590174e+09,-2.92,-39.92,81.27,86.0,100.0,11.59
550,kondinskoye,RU,1.590174e+09,59.65,67.41,49.93,82.0,0.0,8.10
551,poum,NC,1.590174e+09,-20.23,164.02,74.52,73.0,81.0,17.90
552,mahibadhoo,MV,1.590174e+09,3.78,72.97,84.65,78.0,98.0,15.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
!jupyter nbextension list
!jupyter nbextension enable --py gmaps
from ipywidgets.embed import embed_minimal_html
#
gmaps.configure(api_key = g_key)
locations = df[['Lat', 'Lng']]
weights = df['Humidity']
fig = gmaps.figure(map_type = 'HYBRID', center = (0,0), zoom_level = 3)
heat = gmaps.heatmap_layer(locations, weights = weights, max_intensity = 0, point_radius = 30)
fig.add_layer(heat)
#
fig

Known nbextensions:
  config dir: C:\Users\chris\.jupyter\nbconfig
    notebook section
      jupyter-gmaps/extension enabled 
  config dir: C:\Users\chris\Anaconda3\etc\jupyter\nbconfig
    notebook section
      jupyter-js-widgets/extension enabled 


      - Validating: ok
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
hotel_df = df.loc[(df["Max Temp"] <= 80) & (df["Max Temp"] >= 70) &(df["Cloudiness"] == 0) &(df["Wind Speed"] < 10)]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df['Hotel Name'] = ''

C:\Users\chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
params = {
    "radius": 5000, 
    "types": "lodging",
    "key": g_key}

for index, row in hotel_df.iterrows():
    city = row['City']
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params = params)
    response = response.json()
    try:
        print(f'Finding Lodging for {city}')
        print('----------------------------')
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("No Lodging Found.")
        hotel_df.loc[index, "Hotel Name"] = "No Lodging Found"
        

Finding Lodging for farah
----------------------------
No Lodging Found.


C:\Users\chris\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Finding Lodging for korla
----------------------------
Finding Lodging for caravelas
----------------------------
Finding Lodging for mehran
----------------------------
Finding Lodging for sumbe
----------------------------
Finding Lodging for sao filipe
----------------------------
Finding Lodging for gat
----------------------------
Finding Lodging for ambilobe
----------------------------
Finding Lodging for komsomolskiy
----------------------------
Finding Lodging for mattru
----------------------------
Finding Lodging for tazmalt
----------------------------
Finding Lodging for hornepayne
----------------------------


In [9]:
hotel_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
65,farah,AF,1.590174e+09,32.50,62.50,75.67,19.0,0.0,4.38,No Lodging Found
70,korla,CN,1.590174e+09,41.76,86.15,71.56,25.0,0.0,6.55,Gulisitan Hotel
95,caravelas,BR,1.590174e+09,-17.71,-39.25,78.53,74.0,0.0,5.82,Pousada dos Navegantes
116,mehran,IR,1.590174e+09,33.12,46.16,71.60,26.0,0.0,8.99,زائرسرای نور
250,sumbe,AO,1.590174e+09,-11.21,13.84,77.52,85.0,0.0,6.80,Hotel Ritz Sumbe
279,sao filipe,CV,1.590174e+09,14.90,-24.50,75.99,69.0,0.0,8.63,Tortuga B&B
285,gat,IL,1.590174e+09,31.61,34.76,75.00,75.0,0.0,7.20,רגע בנחת
357,ambilobe,MG,1.590174e+09,-13.20,49.05,71.29,79.0,0.0,2.06,Hôtel & Restaurant DIANA Ambilobe
358,komsomolskiy,UZ,1.590174e+09,40.43,71.72,73.40,48.0,0.0,2.24,"Hotel ""Ikathouse"""
432,mattru,SL,1.590174e+09,7.62,-11.83,78.60,93.0,0.0,3.38,Boima Gbondo's Compound


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
lodging = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
# Display figure
hotel_plot = gmaps.symbol_layer(lodging, fill_color = 'red', stroke_color = 'blue', info_box_content = hotel_info)
fig.add_layer(hotel_plot)
#
fig

Figure(layout=FigureLayout(height='420px'))